In [ ]:
print("K")

: 

In [6]:
print("K")
import ee
import geemap
import numpy as np
import rasterio
import matplotlib.pyplot as plt
ee.Authenticate()

# Initialize Earth Engine
ee.Initialize(project='agb-sarbanibhadra')
# Define the Area of Interest (AOI)
amazon_basin = ee.Geometry.Polygon([
  [
      [-80.0, -25.0],
      [-40.0, -25.0],
      [-40.0, 5.0],
      [-80.0, 5.0]
  ]
])
# Load Sentinel-1 data
sentinel1 = ee.ImageCollection('COPERNICUS/S1_GRD') \
    .filterBounds(amazon_basin) \
    .filterDate('2020-01-01', '2020-12-31') \
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
    .select('VV')

sentinel1_median = sentinel1.median().clip(amazon_basin).resample('bilinear').reproject(crs='EPSG:4326', scale=10)
print(sentinel1_median)

# Load Sentinel-2 data
sentinel2 = ee.ImageCollection('COPERNICUS/S2_HARMONIZED') \
    .filterBounds(amazon_basin) \
    .filterDate('2020-01-01', '2020-12-31') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
    .select(['B4', 'B3', 'B2'])  # Red, Green, Blue bands

sentinel2_median = sentinel2.median().clip(amazon_basin).resample('bilinear').reproject(crs='EPSG:4326', scale=10)
print(sentinel2_median)

# Load DEM data
dem = ee.Image('USGS/SRTMGL1_003').clip(amazon_basin).resample('bilinear').reproject(crs='EPSG:4326', scale=10)

# Compute slope from DEM
slope = ee.Terrain.slope(dem)
aspect = ee.Terrain.aspect(dem)

# Combine features into a single image
combined_image = sentinel1_median.addBands(sentinel2_median).addBands(dem).addBands(slope).addBands(aspect)


agb_coefficients = [0.1, 0.2, 0.3, 0.4, 0.05, 0.05]  

agb = combined_image.expression(
    'b(0) * coef0 + b(1) * coef1 + b(2) * coef2 + b(3) * coef3 + b(4) * coef4 + b(5) * coef5',
    {
        'coef0': agb_coefficients[0],
        'coef1': agb_coefficients[1],
        'coef2': agb_coefficients[2],
        'coef3': agb_coefficients[3],
        'coef4': agb_coefficients[4],
        'coef5': agb_coefficients[5]
    }
)

# redd_projects = ee.FeatureCollection('users/yourusername/yourassetname')

Map = geemap.Map(center=[-2, -60], zoom=5)


Map.addLayer(amazon_basin, {}, 'Amazon Basin')
# Map.addLayer(sentinel2_median, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}, 'Sentinel-2')
Map.addLayer(agb, {'min': 0, 'max': 500, 'palette': ['blue', 'green', 'yellow', 'red']}, 'AGB')


# map.addLayer(redd_projects, {
#     'color': 'red',
#     'opacity': 0.5
# }, 'REDD+ Projects')


# Add layer controls
Map.addLayerControl()

# map.save('agb_map_with_redd_projects.png')

print("done")



K
ee.Image({
  "functionInvocationValue": {
    "functionName": "Image.reproject",
    "arguments": {
      "crs": {
        "functionInvocationValue": {
          "functionName": "Projection",
          "arguments": {
            "crs": {
              "constantValue": "EPSG:4326"
            }
          }
        }
      },
      "image": {
        "functionInvocationValue": {
          "functionName": "Image.resample",
          "arguments": {
            "image": {
              "functionInvocationValue": {
                "functionName": "Image.clip",
                "arguments": {
                  "geometry": {
                    "functionInvocationValue": {
                      "functionName": "GeometryConstructors.Polygon",
                      "arguments": {
                        "coordinates": {
                          "constantValue": [
                            [
                              [
                                -80.0,
                               

In [7]:
Map

Map(center=[-2, -60], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(chi…